# Dataset 다운받기

In [ ]:
!curl -L "https://universe.roboflow.com/ds/hGiLinRKJM?key=K4uZJ8TNHq" > Car_damage_2.zip; #unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   903  100   903    0     0   2215      0 --:--:-- --:--:-- --:--:--  2218
100 23.1M  100 23.1M    0     0  5771k      0  0:00:04  0:00:04 --:--:-- 9495k


구글 마운트해서 내 드라이브에 저장

In [ ]:
#구글 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#루트 디렉토리 설정
root_dir = '/content/drive/MyDrive/Colab_Notebooks/likelion_AIEngineer/Car_Damage_Detection/Car_Damage_2_data'

In [ ]:
#/content에 받은 zip파일 루트에 풀고 zip파일 지우기
!unzip -q Car_damage_2.zip -d {root_dir}; rm Car_damage_2.zip

In [ ]:
import os
import cv2
import json
from collections import defaultdict
import shutil
from glob import glob
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [ ]:
root_dir

'/content/drive/MyDrive/Colab_Notebooks/likelion_AIEngineer/Car_Damage_Detection/Car_Damage_2_data'

In [ ]:
#1. train 데이터 로드
train_image_dir = f'{root_dir}/train/images/'
train_label_dir = f'{root_dir}/train/labels/'

train_images = sorted(glob(os.path.join(train_image_dir, '*.jpg')))
train_labels = sorted(glob(os.path.join(train_label_dir, '*.txt')))

print("\n=== 파일 개수 ===")
print(f'이미지 개수: {len(train_images)}, 라벨 개수: {len(train_labels)}')


=== 파일 개수 ===
이미지 개수: 1437, 라벨 개수: 1437


In [ ]:
# 2. 클래스 분포 확인
#{0: Scatch(긁힘), 1: Dent(찍힘), 2: Crush(찌그러짐), 3: Car_damage}
damage_class = {0:0, 1:0, 2:0, 3:0}
class_mapping = {
    0 : 'Scatch(긁힘)',
    1 : 'Dent(찍힘)',
    2 : 'Crash(찌그러짐)',
    3 : 'Car_damage'
}

for label in train_labels:
  with open(label, 'r') as f:
      lines = f.readlines()
      for line in lines:
          class_id = int(line.split()[0])
          damage_class[class_id] += 1

print("\n=== 데이터 분포 ===")
total = sum(damage_class.values())
for damage_type, count in damage_class.items():
    korean_name = class_mapping.get(damage_type, damage_type)
    percentage = (count / total) * 100
    print(f"{damage_type} {korean_name}: {count} ({percentage:.1f}%)")



=== 데이터 분포 ===
0 Scatch(긁힘): 1396 (50.5%)
1 Dent(찍힘): 563 (20.3%)
2 Crash(찌그러짐): 400 (14.5%)
3 Car_damage: 408 (14.7%)


In [ ]:
# 3. YOLO 형식의 데이터셋 구조 생성
# train_split directory, validation directory 만들기

train_split_image_dir = f'{root_dir}/train_split/images/'
train_split_label_dir = f'{root_dir}/train_split/labels/'
val_image_dir = f'{root_dir}/val/images/'
val_label_dir = f'{root_dir}/val/labels/'

os.makedirs(train_split_image_dir, exist_ok=True)
os.makedirs(train_split_label_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

In [ ]:
# 4. 데이터 분할 및 변환
#train 데이터를 train과 val로 분할 (train:val:test = 0.7:0.27:0.03)
from sklearn.model_selection import train_test_split

train_split_images, val_images = train_test_split(train_images, train_size=0.7, random_state=42)

In [ ]:
# 분할된 데이터셋 처리
splits = {
    'train_split': train_split_images,
    'val': val_images,
}
cls_list = [0, 1, 2, 3]
for split_name, split_images in splits.items():
  print(f"\n처리 중: {split_name} 셋")
  for img_path in split_images:
    #이미지 복사
    img_filename = os.path.basename(img_path)
    shutil.copy(img_path, f'{root_dir}/{split_name}/images/{img_filename}')

    #라벨 복사
    label_filename = os.path.splitext(img_filename)[0] + '.txt'
    shutil.copy(os.path.join(train_label_dir, label_filename), f'{root_dir}/{split_name}/labels/{label_filename}')

print("\n데이터셋 변환 완료")
print(f"Train Split 이미지: {len(train_split_images)}")
print(f"Validation 이미지: {len(val_images)}")



처리 중: train_split 셋

처리 중: val 셋

데이터셋 변환 완료
Train Split 이미지: 1005
Validation 이미지: 432


# Dataset에서 class 3(Car_damage) 제거

In [ ]:
# 5. class 3(Car_damage)제거를 위한 디렉토리 생성
# train_split_del dir, validation_del dir, test_del dir 만들기

train_split_del_image_dir = f'{root_dir}/train_split_del/images/'
train_split_del_label_dir = f'{root_dir}/train_split_del/labels/'
val_del_image_dir = f'{root_dir}/val_del/images/'
val_del_label_dir = f'{root_dir}/val_del/labels/'
test_del_image_dir = f'{root_dir}/test_del/images/'
test_del_label_dir = f'{root_dir}/test_del/labels/'

os.makedirs(train_split_del_image_dir, exist_ok=True)
os.makedirs(train_split_del_label_dir, exist_ok=True)
os.makedirs(val_del_image_dir, exist_ok=True)
os.makedirs(val_del_label_dir, exist_ok=True)
os.makedirs(test_del_image_dir, exist_ok=True)
os.makedirs(test_del_label_dir, exist_ok=True)

In [ ]:
#6. 데이터 로드
#train 데이터 로드
train_split_image_dir = f'{root_dir}/train_split/images/'
train_split_label_dir = f'{root_dir}/train_split/labels/'

train_split_images = sorted(glob(os.path.join(train_split_image_dir, '*.jpg')))
train_split_labels = sorted(glob(os.path.join(train_split_label_dir, '*.txt')))

#val 데이터 로드
val_image_dir = f'{root_dir}/val/images/'
val_label_dir = f'{root_dir}/val/labels/'

val_images = sorted(glob(os.path.join(val_image_dir, '*.jpg')))
val_labels = sorted(glob(os.path.join(val_label_dir, '*.txt')))

#test 데이터 로드
test_image_dir = f'{root_dir}/test/images/'
test_label_dir = f'{root_dir}/test/labels/'

test_images = sorted(glob(os.path.join(test_image_dir, '*.jpg')))
test_labels = sorted(glob(os.path.join(test_label_dir, '*.txt')))


print("\n=== 파일 개수 ===")
print(f'이미지 개수: {len(train_split_images)}, 라벨 개수: {len(train_split_labels)}')
print(f'이미지 개수: {len(val_images)}, 라벨 개수: {len(val_labels)}')
print(f'이미지 개수: {len(test_images)}, 라벨 개수: {len(test_labels)}')


=== 파일 개수 ===
이미지 개수: 1005, 라벨 개수: 1005
이미지 개수: 432, 라벨 개수: 432
이미지 개수: 49, 라벨 개수: 49


In [ ]:
#7. 각각 디렉토리에 데이터셋 복사(백업하려고)
splits = {
    'train_split_del': train_split_images,
    'val_del': val_images,
    'test_del': test_images
}

for split_name, split_images in splits.items():
  print(f"\n처리 중: {split_name} 셋")
  for img_path in split_images:
    #이미지 복사
    img_filename = os.path.basename(img_path)
    shutil.copy(img_path, f'{root_dir}/{split_name}/images/{img_filename}')

    #라벨 복사
    label_path = os.path.dirname(img_path.split('images')[0])
    label_filename = os.path.splitext(img_filename)[0] + '.txt'
    shutil.copy(f'{label_path}/labels/{label_filename}', f'{root_dir}/{split_name}/labels/{label_filename}')


처리 중: train_split_del 셋

처리 중: val_del 셋

처리 중: test_del 셋


In [ ]:
#처리할 데이터셋 로드
#train 데이터 로드
train_split_del_images = sorted(glob(f'{root_dir}/train_split_del/images/*.jpg'))
train_split_del_labels = sorted(glob(f'{root_dir}/train_split_del/labels/*.txt'))

#val 데이터 로드
val_del_images = sorted(glob(f'{root_dir}/val_del/images/*.jpg'))
val_del_labels = sorted(glob(f'{root_dir}/val_del/labels/*.txt'))

#test 데이터 로드
test_del_images = sorted(glob(f'{root_dir}/test_del/images/*.jpg'))
test_del_labels = sorted(glob(f'{root_dir}/test_del/labels/*.txt'))

print("\n데이터셋 로드 완료")
print(f"Train Split del 이미지: {len(train_split_del_images)}")
print(f"Validation del 이미지: {len(val_del_images)}")
print(f"Test del 이미지: {len(test_del_images)}")


데이터셋 로드 완료
Train Split del 이미지: 1005
Validation del 이미지: 432
Test del 이미지: 49


- 클래스 3 분포 확인
  - 클래스 3만 있는 이미지데이터는 삭제
  - 클래스 3이 포함되어있는 이미지데이터는 라벨데이터에서 클래스3만 삭제
  - 클래스 3이 포함되어있지 않은 이미지 데이터는 그냥 넘어감

In [ ]:
#8. 클래스3 분포확인하고 삭제 처리
splits = {
    'train_split_del': train_split_del_labels,
    'val_del': val_del_labels,
    'test_del': test_del_labels
}
class3_label_files = {'all_3':0, 'any_3':0, 'no_3':0}
class3_all = list()

for split_name, split_labels in splits.items():
  print(f"\n처리 중: {split_name} 셋")
  for label_path in split_labels:
    with open(label_path, 'r') as f:
      lines = f.readlines()

      # class3만 있는지 확인
      if all(int(line.split()[0]) == 3 for line in lines):
        class3_label_files['all_3'] += 1
        class3_all.append(label_path)

        #이미지 경로 설정 및 파일 삭제
        label_filename = os.path.basename(label_path)
        temp_path = os.path.dirname(label_path.split('labels')[0])
        image_filename = os.path.splitext(label_filename)[0] + '.jpg'
        image_path = os.path.join(temp_path, 'images', image_filename)

        # 파일 존재 확인 후 삭제
        if os.path.exists(label_path):
            os.remove(label_path)
        if os.path.exists(image_path):
            os.remove(image_path)

      # class3이 있는지 확인, 모든 줄이 class 3은 아님
      elif any(int(line.split()[0]) == 3 for line in lines) and not all(int(line.split()[0]) == 3 for line in lines):
        class3_label_files['any_3'] += 1
        # class 3이 아닌 라벨만 다시 쓰기
        with open(label_path, 'w') as f:
          for line in lines:
              if int(line.split()[0]) != 3:
                  f.write(line)
        print(f"class 3 라벨 제거: {label_path[:-4] + '.jpg'}")

      #class3 없는거
      else:
        class3_label_files['no_3'] += 1

print("\n=== 클래스 3 분포 ===")
print(f"클래스 3만 있는 파일 수: {class3_label_files['all_3']}")
print(f"클래스 3 있는 파일 수(class 3만은 제외): {class3_label_files['any_3']}")
print(f"클래스 3 없는 파일 수: {class3_label_files['no_3']}")


처리 중: train_split_del 셋

처리 중: val_del 셋

처리 중: test_del 셋

=== 클래스 3 분포 ===
클래스 3만 있는 파일 수: 281
클래스 3 있는 파일 수(class 3만은 제외): 0
클래스 3 없는 파일 수: 1205


In [ ]:
#처리 후 데이터 수 확인
#train 데이터 로드
train_split_del_images = sorted(glob(f'{root_dir}/train_split_del/images/*.jpg'))
train_split_del_labels = sorted(glob(f'{root_dir}/train_split_del/labels/*.txt'))

#val 데이터 로드
val_del_images = sorted(glob(f'{root_dir}/val_del/images/*.jpg'))
val_del_labels = sorted(glob(f'{root_dir}/val_del/labels/*.txt'))

#test 데이터 로드
test_del_images = sorted(glob(f'{root_dir}/test_del/images/*.jpg'))
test_del_labels = sorted(glob(f'{root_dir}/test_del/labels/*.txt'))

print("\n=== 처리 후 데이터 갯수 ===")
print(f"Train Split del 이미지: {len(train_split_del_images)}")
print(f"Validation del 이미지: {len(val_del_images)}")
print(f"Test del 이미지: {len(test_del_images)}")


=== 처리 후 데이터 갯수 ===
Train Split del 이미지: 817
Validation del 이미지: 347
Test del 이미지: 41


In [ ]:
# 9. 삭제 후 클래스 분포 확인
#{0: Scatch(긁힘), 1: Dent(찍힘), 2: Crush(찌그러짐), 3: Car_damage}
damage_class = {0:0, 1:0, 2:0, 3:0}
class_mapping = {
    0 : 'Scatch(긁힘)',
    1 : 'Dent(찍힘)',
    2 : 'Crash(찌그러짐)',
    3 : 'Car_damage'
}

for label in train_split_del_labels:
  with open(label, 'r') as f:
      lines = f.readlines()
      for line in lines:
          class_id = int(line.split()[0])
          damage_class[class_id] += 1

print("\n=== 데이터 분포 ===")
total = sum(damage_class.values())
for damage_type, count in damage_class.items():
    korean_name = class_mapping.get(damage_type, damage_type)
    percentage = (count / total) * 100
    print(f"{damage_type} {korean_name}: {count} ({percentage:.1f}%)")


=== 데이터 분포 ===
0 Scatch(긁힘): 977 (59.4%)
1 Dent(찍힘): 385 (23.4%)
2 Crash(찌그러짐): 282 (17.2%)
3 Car_damage: 0 (0.0%)


In [ ]:
# 10. YAML 설정 파일 생성
cls_list = [0, 1, 2, 3]
del_3_cls_list = [0, 1, 2]  # Removed class 3
yaml_content = f"""
path: {root_dir}
train: train_split_del/images
val: val_del/images
test: test_del/images

nc: {len(del_3_cls_list)}  # Number of classes set to 3
names: {del_3_cls_list}  # Class names set to [0, 1, 2]
"""

yaml_path = os.path.join(root_dir, 'data_del_class3.yaml')
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print("\nYAML 설정 파일 생성 완료")


YAML 설정 파일 생성 완료
